In [1]:
%run PROMETHEE_funct.ipynb

Pixiedust database opened successfully


In [2]:
tabel_alternatif = open_tabel("Tabel Alternatif.csv")

tabel_alternatif

,Administrasi Fiskal - UI (IPS),Administrasi Negara - UNPAD (IPS),Administrasi Niaga - UI (IPS),Administrasi Perpajakan - UB (IPS),Agribisnis - IPB (IPA),Agroekoteknologi - UB (IPA),Agronomi Dan Holtikultura - IPB (IPA),Agroteknologi - UNPAD (IPA),Akuntansi - UGM (IPS),Akutansi - UI (IPS),...,Teknologi Dan Manajemen Perikanan Budidaya - IPB (IPS),Teknologi Hasil Perikanan - UB (IPA),Teknologi Industri Hasil Perikanan - UNAIR (IPA),Teknologi Industri Pertanian - IPB (IPA),Teknologi Informasi - UGM (IPA),Teknologi Pangan - IPB (IPA),Televisi & Film - UNPAD (IPS),Tin - IPB (IPA),Tip - UGM (IPA),Tv & Film - UNPAD (IPS)
Matematika,84.000000,88.500000,85.465625,92.000000,95.000,81.852143,91.000000,82.500000,91.625000,44.693125,...,84.500000,82.000000,85.000000,85.410699,92.000000,92.000000,85.000000,90.00,87.000000,85.000000
Inggris,84.000000,80.500000,83.888667,86.000000,90.000,84.729722,91.000000,82.750000,90.875000,54.255467,...,85.500000,91.000000,84.000000,84.775000,90.000000,88.000000,85.500000,83.00,94.000000,85.000000
Indonesia,84.000000,89.500000,85.246032,89.000000,88.000,85.837917,89.000000,84.000000,89.875000,85.000000,...,86.500000,89.000000,86.000000,86.789540,93.000000,86.000000,86.500000,89.00,80.000000,84.000000
Fisika,82.000000,84.584430,84.493016,81.956615,91.000,81.739154,89.000000,80.500000,89.437576,87.000000,...,84.500000,80.000000,83.000000,85.810345,89.000000,92.000000,84.584430,88.00,86.000000,84.584430
Kimia,81.000000,84.592411,85.297674,82.397782,89.000,82.349446,90.000000,81.500000,88.716106,74.300000,...,84.000000,79.000000,81.000000,84.825000,89.000000,85.000000,84.592411,88.00,87.000000,84.592411
Biologi,83.000000,83.879942,84.182558,82.827536,90.000,83.706884,91.000000,83.250000,89.695798,83.000000,...,84.000000,83.000000,86.000000,86.391667,93.000000,86.000000,83.879942,89.00,86.000000,83.879942
Ekonomi,86.145714,89.000000,86.145714,91.242143,85.000,83.726429,83.000000,82.062045,92.250000,86.145714,...,84.422980,88.726429,85.211111,84.961490,89.837895,85.845960,86.000000,83.00,89.837895,86.000000
Geografi,85.512727,87.500000,85.512727,91.000000,81.316,84.627747,83.526667,81.926031,88.959803,85.512727,...,85.388333,84.808608,84.793706,83.645000,88.094322,87.250000,86.500000,87.25,88.094322,87.000000
Sosiologi,86.930877,85.500000,86.930877,91.000000,NaN,85.702500,NaN,81.331846,91.872475,86.930877,...,NaN,84.850000,85.695971,NaN,90.363636,NaN,85.500000,NaN,90.363636,85.000000
Sejarah,83.943883,88.000000,83.943883,93.000000,89.000,85.517917,82.676471,81.000669,90.581317,85.000000,...,85.958333,83.000000,84.000000,85.088235,91.000000,85.916667,86.500000,88.00,88.650538,85.000000


In [3]:
nilai_jams = [70, 70, 70, 70, 70, 70, 70, 70, 70, 70]

In [4]:
mean = get_mean(tabel_alternatif)
std = get_std(tabel_alternatif)
params = get_params(mean, nilai_jams, std)

params

[-2.9962586995971607,
 -3.7659282376213357,
 -4.836058445209642,
 -3.88743687886157,
 -3.9444976888163192,
 -5.01053272698281,
 -5.899184296964397,
 -6.429658422651344,
 -5.644181966720148,
 -5.287529145482948]

In [5]:
tabel_alternatif = add_parameter(params, tabel_alternatif)

tabel_alternatif

,Administrasi Fiskal - UI (IPS),Administrasi Negara - UNPAD (IPS),Administrasi Niaga - UI (IPS),Administrasi Perpajakan - UB (IPS),Agribisnis - IPB (IPA),Agroekoteknologi - UB (IPA),Agronomi Dan Holtikultura - IPB (IPA),Agroteknologi - UNPAD (IPA),Akuntansi - UGM (IPS),Akutansi - UI (IPS),...,Teknologi Hasil Perikanan - UB (IPA),Teknologi Industri Hasil Perikanan - UNAIR (IPA),Teknologi Industri Pertanian - IPB (IPA),Teknologi Informasi - UGM (IPA),Teknologi Pangan - IPB (IPA),Televisi & Film - UNPAD (IPS),Tin - IPB (IPA),Tip - UGM (IPA),Tv & Film - UNPAD (IPS),Parameter
Matematika,84.000000,88.500000,85.465625,92.000000,95.000,81.852143,91.000000,82.500000,91.625000,44.693125,...,82.000000,85.000000,85.410699,92.000000,92.000000,85.000000,90.00,87.000000,85.000000,-2.996259
Inggris,84.000000,80.500000,83.888667,86.000000,90.000,84.729722,91.000000,82.750000,90.875000,54.255467,...,91.000000,84.000000,84.775000,90.000000,88.000000,85.500000,83.00,94.000000,85.000000,-3.765928
Indonesia,84.000000,89.500000,85.246032,89.000000,88.000,85.837917,89.000000,84.000000,89.875000,85.000000,...,89.000000,86.000000,86.789540,93.000000,86.000000,86.500000,89.00,80.000000,84.000000,-4.836058
Fisika,82.000000,84.584430,84.493016,81.956615,91.000,81.739154,89.000000,80.500000,89.437576,87.000000,...,80.000000,83.000000,85.810345,89.000000,92.000000,84.584430,88.00,86.000000,84.584430,-3.887437
Kimia,81.000000,84.592411,85.297674,82.397782,89.000,82.349446,90.000000,81.500000,88.716106,74.300000,...,79.000000,81.000000,84.825000,89.000000,85.000000,84.592411,88.00,87.000000,84.592411,-3.944498
Biologi,83.000000,83.879942,84.182558,82.827536,90.000,83.706884,91.000000,83.250000,89.695798,83.000000,...,83.000000,86.000000,86.391667,93.000000,86.000000,83.879942,89.00,86.000000,83.879942,-5.010533
Ekonomi,86.145714,89.000000,86.145714,91.242143,85.000,83.726429,83.000000,82.062045,92.250000,86.145714,...,88.726429,85.211111,84.961490,89.837895,85.845960,86.000000,83.00,89.837895,86.000000,-5.899184
Geografi,85.512727,87.500000,85.512727,91.000000,81.316,84.627747,83.526667,81.926031,88.959803,85.512727,...,84.808608,84.793706,83.645000,88.094322,87.250000,86.500000,87.25,88.094322,87.000000,-6.429658
Sosiologi,86.930877,85.500000,86.930877,91.000000,NaN,85.702500,NaN,81.331846,91.872475,86.930877,...,84.850000,85.695971,NaN,90.363636,NaN,85.500000,NaN,90.363636,85.000000,-5.644182
Sejarah,83.943883,88.000000,83.943883,93.000000,89.000,85.517917,82.676471,81.000669,90.581317,85.000000,...,83.000000,84.000000,85.088235,91.000000,85.916667,86.500000,88.00,88.650538,85.000000,-5.287529


In [6]:
pilihan_jurusan = get_jurusan(tabel_alternatif)

pilihan_jurusan

['Administrasi Fiskal - UI (IPS)',
 'Administrasi Negara - UNPAD (IPS)',
 'Administrasi Niaga - UI (IPS)',
 'Administrasi Perpajakan - UB (IPS)',
 'Agribisnis - IPB (IPA)',
 'Agroekoteknologi - UB (IPA)',
 'Agronomi Dan Holtikultura - IPB (IPA)',
 'Agroteknologi - UNPAD (IPA)',
 'Akuntansi - UGM (IPS)',
 'Akutansi - UI (IPS)',
 'Antropologi - UNAIR (IPA)',
 'Antropologi Sosial - UB (IPS)',
 'Apoteker - UNAIR (IPA)',
 'Arkeologi - UI (IPA)',
 'Arsitektur - ITB (IPA)',
 'Arsitektur Lansekap - IPB (IPA)',
 'Astronomi - ITB (IPA)',
 'Biokimia - IPB (IPA)',
 'Biologi - ITS (IPA)',
 'Budidaya Perairan - UNAIR (IPA)',
 'Desain Interior - ITS (IPS)',
 'Ekonomi Dan Studi Pembangunan - IPB (IPS)',
 'Ekonomi Islam - UB (IPS)',
 'Ekonomi Islam  - UNPAD (IPS)',
 'Ekonomi Pembangunan - IPB (IPS)',
 'Elektro - ITB (IPA)',
 'Fakultas Kedokteran Gigi - UNAIR (IPA)',
 'Fakultas Kedokteran Hewan - IPB (IPA)',
 'Fakultas Teknik Industri - ITB (IPA)',
 'Farmasi - UI (IPA)',
 'Fisika - IPB (IPA)',
 'Fiskal 

In [7]:
fp_gaussian = fungsi_preferensi_gaussian(tabel_alternatif)

fp_gaussian

,Administrasi Fiskal - UI (IPS)&Administrasi Negara - UNPAD (IPS),Administrasi Fiskal - UI (IPS)&Administrasi Niaga - UI (IPS),Administrasi Fiskal - UI (IPS)&Administrasi Perpajakan - UB (IPS),Administrasi Fiskal - UI (IPS)&Agribisnis - IPB (IPA),Administrasi Fiskal - UI (IPS)&Agroekoteknologi - UB (IPA),Administrasi Fiskal - UI (IPS)&Agronomi Dan Holtikultura - IPB (IPA),Administrasi Fiskal - UI (IPS)&Agroteknologi - UNPAD (IPA),Administrasi Fiskal - UI (IPS)&Akuntansi - UGM (IPS),Administrasi Fiskal - UI (IPS)&Akutansi - UI (IPS),Administrasi Fiskal - UI (IPS)&Antropologi - UNAIR (IPA),...,Tv & Film - UNPAD (IPS)&Teknologi Bioproses - UI (IPA),Tv & Film - UNPAD (IPS)&Teknologi Dan Manajemen Perikanan Budidaya - IPB (IPS),Tv & Film - UNPAD (IPS)&Teknologi Hasil Perikanan - UB (IPA),Tv & Film - UNPAD (IPS)&Teknologi Industri Hasil Perikanan - UNAIR (IPA),Tv & Film - UNPAD (IPS)&Teknologi Industri Pertanian - IPB (IPA),Tv & Film - UNPAD (IPS)&Teknologi Informasi - UGM (IPA),Tv & Film - UNPAD (IPS)&Teknologi Pangan - IPB (IPA),Tv & Film - UNPAD (IPS)&Televisi & Film - UNPAD (IPS),Tv & Film - UNPAD (IPS)&Tin - IPB (IPA),Tv & Film - UNPAD (IPS)&Tip - UGM (IPA)
Matematika,0.000000,0.000000,0,0.000000,0.000000,0.000000,0.000000,0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.000000,0.000000
Inggris,0.350712,0.000437,0,0.000000,0.000000,0.000000,0.053597,0,1.0,0.000000,...,0.042608,0.000000,0.000000,0.034641,0.001783,0,0.000000,0.000000,0.131530,0.000000
Indonesia,0.000000,0.000000,0,0.000000,0.000000,0.000000,0.000000,0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.000000,0.289697
Fisika,0.000000,0.000000,0,0.000000,0.000000,0.000000,0.000000,0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.000000,0.000000
Kimia,0.000000,0.000000,0,0.000000,0.000000,0.000000,0.000000,0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.000000,0.000000
Biologi,0.000000,0.000000,0,0.000000,0.000000,0.000000,0.000000,0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.000000,0.000000
Ekonomi,0.000000,0.000000,0,0.018683,0.080654,0.132531,0.213058,0,0.0,0.012471,...,0.000000,0.035101,0.000000,0.008902,0.015376,0,0.000341,0.000000,0.121298,0.000000
Geografi,0.000000,0.000000,0,0.191858,0.009428,0.046587,0.144091,0,0.0,0.006233,...,0.026398,0.030927,0.056427,0.057174,0.127278,0,0.000000,0.003019,0.000000,0.000000
Sosiologi,0.031624,0.000000,0,0.000000,0.023404,0.000000,0.388617,0,0.0,0.023651,...,0.000000,0.000000,0.000353,0.000000,0.000000,0,0.000000,0.000000,0.000000,0.000000
Sejarah,0.000000,0.000000,0,0.000000,0.000000,0.028319,0.143517,0,0.0,0.001400,...,0.019750,0.000000,0.069037,0.017725,0.000000,0,0.000000,0.000000,0.000000,0.000000


In [8]:
index_preferensi_global = idx_pref_global(fp_gaussian)

index_preferensi_global

,Administrasi Fiskal - UI (IPS)&Administrasi Negara - UNPAD (IPS),Administrasi Fiskal - UI (IPS)&Administrasi Niaga - UI (IPS),Administrasi Fiskal - UI (IPS)&Administrasi Perpajakan - UB (IPS),Administrasi Fiskal - UI (IPS)&Agribisnis - IPB (IPA),Administrasi Fiskal - UI (IPS)&Agroekoteknologi - UB (IPA),Administrasi Fiskal - UI (IPS)&Agronomi Dan Holtikultura - IPB (IPA),Administrasi Fiskal - UI (IPS)&Agroteknologi - UNPAD (IPA),Administrasi Fiskal - UI (IPS)&Akuntansi - UGM (IPS),Administrasi Fiskal - UI (IPS)&Akutansi - UI (IPS),Administrasi Fiskal - UI (IPS)&Antropologi - UNAIR (IPA),...,Tv & Film - UNPAD (IPS)&Teknologi Bioproses - UI (IPA),Tv & Film - UNPAD (IPS)&Teknologi Dan Manajemen Perikanan Budidaya - IPB (IPS),Tv & Film - UNPAD (IPS)&Teknologi Hasil Perikanan - UB (IPA),Tv & Film - UNPAD (IPS)&Teknologi Industri Hasil Perikanan - UNAIR (IPA),Tv & Film - UNPAD (IPS)&Teknologi Industri Pertanian - IPB (IPA),Tv & Film - UNPAD (IPS)&Teknologi Informasi - UGM (IPA),Tv & Film - UNPAD (IPS)&Teknologi Pangan - IPB (IPA),Tv & Film - UNPAD (IPS)&Televisi & Film - UNPAD (IPS),Tv & Film - UNPAD (IPS)&Tin - IPB (IPA),Tv & Film - UNPAD (IPS)&Tip - UGM (IPA)
Index Preferensi Global,0.152934,0.000175,0.0,0.084217,0.045395,0.082975,0.377152,0.0,0.4,0.017502,...,0.035503,0.019809,0.037745,0.035533,0.043331,0.0,0.000102,0.001208,0.101131,0.115879


In [9]:
e_flow = entering_flow(pilihan_jurusan, index_preferensi_global)

e_flow

{'Administrasi Fiskal - UI (IPS)': 0.7413379793232232,
 'Administrasi Negara - UNPAD (IPS)': 0.4792838147693877,
 'Administrasi Niaga - UI (IPS)': 0.4668650920876039,
 'Administrasi Perpajakan - UB (IPS)': 0.4217179634439663,
 'Agribisnis - IPB (IPA)': 0.08573748283179757,
 'Agroekoteknologi - UB (IPA)': 0.7042280630971399,
 'Agronomi Dan Holtikultura - IPB (IPA)': 0.10968672794937097,
 'Agroteknologi - UNPAD (IPA)': 0.9398081073824245,
 'Akuntansi - UGM (IPS)': 0.050748836644980824,
 'Akutansi - UI (IPS)': 1.14630076799873,
 'Antropologi - UNAIR (IPA)': 0.4240672383741827,
 'Antropologi Sosial - UB (IPS)': 1.7830042461117752,
 'Apoteker - UNAIR (IPA)': 0.2632059708455146,
 'Arkeologi - UI (IPA)': 0.413488495444544,
 'Arsitektur - ITB (IPA)': 0.1670318227822334,
 'Arsitektur Lansekap - IPB (IPA)': 0.4468095887834523,
 'Astronomi - ITB (IPA)': 0.43193499484570125,
 'Biokimia - IPB (IPA)': 0.3188959740519863,
 'Biologi - ITS (IPA)': 0.25853848755886144,
 'Budidaya Perairan - UNAIR (IPA)'

In [10]:
l_flow = leaving_flow(pilihan_jurusan, index_preferensi_global)

l_flow

{'Administrasi Fiskal - UI (IPS)': 0.08439575145647611,
 'Administrasi Negara - UNPAD (IPS)': 0.2892298276474662,
 'Administrasi Niaga - UI (IPS)': 0.0911011831828691,
 'Administrasi Perpajakan - UB (IPS)': 0.6914811531113542,
 'Agribisnis - IPB (IPA)': 1.1335131765717072,
 'Agroekoteknologi - UB (IPA)': 0.09894343409493127,
 'Agronomi Dan Holtikultura - IPB (IPA)': 1.070527900911384,
 'Agroteknologi - UNPAD (IPA)': 0.07080467768858349,
 'Akuntansi - UGM (IPS)': 0.7885670296986559,
 'Akutansi - UI (IPS)': 0.0807234587343075,
 'Antropologi - UNAIR (IPA)': 0.21038276697279384,
 'Antropologi Sosial - UB (IPS)': 0.010700357473406237,
 'Apoteker - UNAIR (IPA)': 0.3581281435844825,
 'Arkeologi - UI (IPA)': 0.2079967368895047,
 'Arsitektur - ITB (IPA)': 0.7601213948496155,
 'Arsitektur Lansekap - IPB (IPA)': 0.7418591375955957,
 'Astronomi - ITB (IPA)': 0.47510666274208296,
 'Biokimia - IPB (IPA)': 0.2904851630922645,
 'Biologi - ITS (IPA)': 0.36405184155721504,
 'Budidaya Perairan - UNAIR (I

In [12]:
n_flow = net_flow(pilihan_jurusan, e_flow, l_flow)

n_flow

{'Administrasi Fiskal - UI (IPS)': -0.656942227866747,
 'Administrasi Negara - UNPAD (IPS)': -0.1900539871219215,
 'Administrasi Niaga - UI (IPS)': -0.3757639089047348,
 'Administrasi Perpajakan - UB (IPS)': 0.2697631896673879,
 'Agribisnis - IPB (IPA)': 1.0477756937399096,
 'Agroekoteknologi - UB (IPA)': -0.6052846290022087,
 'Agronomi Dan Holtikultura - IPB (IPA)': 0.960841172962013,
 'Agroteknologi - UNPAD (IPA)': -0.869003429693841,
 'Akuntansi - UGM (IPS)': 0.737818193053675,
 'Akutansi - UI (IPS)': -1.0655773092644225,
 'Antropologi - UNAIR (IPA)': -0.21368447140138885,
 'Antropologi Sosial - UB (IPS)': -1.772303888638369,
 'Apoteker - UNAIR (IPA)': 0.09492217273896786,
 'Arkeologi - UI (IPA)': -0.20549175855503932,
 'Arsitektur - ITB (IPA)': 0.5930895720673821,
 'Arsitektur Lansekap - IPB (IPA)': 0.29504954881214346,
 'Astronomi - ITB (IPA)': 0.043171667896381716,
 'Biokimia - IPB (IPA)': -0.028410810959721755,
 'Biologi - ITS (IPA)': 0.1055133539983536,
 'Budidaya Perairan - UN

In [32]:
hasil_outrank = sorted(n_flow.items(), key = lambda kv:(kv[1], kv[0]), reverse = True)

tab = dict()

rank = list()
jurusan = list()
net = list()

for i in range(len(hasil_outrank)):
    rank.append(i+1)
    jurusan.append(hasil_outrank[i][0])
    net.append(hasil_outrank[i][1])
    
tab["Jurusan"] = jurusan
tab["Net Flow"] = net

tabel_hasil = pd.DataFrame(tab, index=rank)

pil_satu = tabel_hasil["Jurusan"][1]
pref_jurusan = pil_satu.split(" - ")[1]

pref_jurusan.replace(" (IPA)", "")

'UB'

In [36]:
a = "Administrasi Fiskal - UI (IPS)&Administrasi Negara - UNPAD (IPS)"

b = a.casefold().split("&")[0].replace(" (ips)", "")

"Administrasi fiskal - ui" == b

False